# Attention Mechanism

## How does the attention mechanism work?

*In this example, we focus on Natural Language Processing*

The attention mechanism allows the model to focus on relevant parts of the input data.
It works as follows:

1. **Encoding**

The model receives the input data (e.g. a sentence) and encodes it into a vector representation (embedding).

2. **Query, Key, Value**

The attention mechanism relies on these 3 types of vectors. They are obtained by multiplying the 3 weights matrices (Query, Key, Value) by the encoded input data.

3. **Attention scores**

The scores are obtained by multiplying the Query vector of a single word by the Key vector of all the other words. This gives us the similarity between the word and all the other words.

4. **Softmax**

The softmax function maps the scores to probability making them easily interpretable.

5. **Weighted sum**

The weighted sum is obtained by multiplying the softmax scores by the Value vectors.

*The weighted sum is the output, commonly referred to as the "Context Vector".*

## The General Attention Mechanism

![translation-with-attention](imgs/translation-with-attention.png)
Source: [cloudskillsboost.google](https://www.cloudskillsboost.google/course_sessions/3962322/video/383836)

- ${H}$ is the encoder hidden state at each time step. In other words, the Key vector.
- ${H}(d)$ is the decoder hidden state at each time step. In other words, the Query vector.
- ${a}$ is the attention weights at each time step. In other words, the attention scores.

### Value, Key, Query

**Value vectors**

The Value represent the actual input data. The value vector is the embeddings of each word in the input sentence.

**Key vectors - ${H}$**

They Key is the encoded representation of the input data. Each hidden state generated by the encoder becomes a Key vector.

**Query vectors - ${H}(d)$**

The Query vector is the output of the decoder at each decoding step. 

Let's say we want to translate "I love cats" in french:
- The Value vectors would be the embeddings of the source sentence. The embeddings of "I", "love" and "cats".
- The Key vector would be the encoded sentence, which is nothing other than the hidden states of the encoder. The hidden states of "I", "love" and "cats". 
- The query vector could be the translated word. In our case, it could be the embedding of "J'aime".

### Attention Scores - ${a}$

**Score Formula**:

The attention score between a query vector $Q$ and a key vector $K$ can be obtained using the dot product. Given $Q$ as an $m$-dimensional vector and $K$ as an $n$-dimensional vector, the score (similarity) can be calculated as follows:

$$
\text{Score}(Q, K) = Q \cdot K^T
$$

Mathematical notation:
$$
\text{e}(q, k_i) = q * k_i
$$

**Softmax**

$$
\text{softmax}(S) = \frac{\exp(S)}{\sum{\exp(S)}}
$$

**Weighted sum (Context vector)**

$$
C = \text{softmax}(S) \cdot V
$$

---

### Refresh on matrix multiplication

Before diving into the attention mechanism, let's refresh our memory on matrix multiplication.

- M1 * M2 is the element-wise product of Matrice 1 by Matrice 2
```python
[1, 2] * [5, 6] = [5, 12]
[3, 4]   [7, 8]   [21, 32]
```

- M1 @ M2 is the multiplication of Matrice 1 by Matrice 2
```python
        [5, 6]
        [7, 8]
[1, 2] @       =  [19, 22]
[3, 4]            [43, 50]
```

- np.dot(M1, M2) is the same as M1 @ M2 but is compatible with more types of objects.

In pratice, we often need to multiply matrices all at once, instead of taking each of them one by one:

```python
Q = [2 0 2]        K = [2 2 2]
    [2 0 0]            [0 2 1]
    [4 0 2]            [2 4 3]
    [2 1 2]            [0 1 1]

# We want to multiply each Query by all the Keys (E.g. Q[0] * K)

We could use:
scores[0] = Q[0] @ K[0], Q[0] @ K[1], ...
scores[1] = Q[1] @ K[0], Q[1] @ K[1], ...

print(scores[0])
print(scores[1])

... [8 4 8 2]
... [4 0 4 0]
```

But a more convenient way to do so is to transpose the Keys matrix and multiply it by the Query matrix:

```python
# We transpose K
K.T = [2 0 2 0]
      [2 2 4 1]
      [2 1 3 1]

# We compute Q @ K.T

                [[2 0 2 0]
                [2 2 4 1]
                [2 1 3 1]]
            @
 [2 0 2]        [8 4 8 2]
 [2 0 0]    =   [4 0 4 0]
 [4 0 2]        [8 4 8 2]
 [2 1 2]        [6 2 6 2]               
````

---

## Practice

In this example, we have four words as input, the goal is to calculate their attention weights.

In [73]:
import numpy as np
from scipy.special import softmax

np.random.seed(42)

### Generate data

We are going to generate some fake data:
1. Embeddings of 4 words generated by the encoder
2. Weights matrices of Query, Key, Value for each of the 4 words

**1. Generate the encoder representations (embeddings) of the words**

In [74]:
word_1 = np.array([1, 0, 0])
word_2 = np.array([0, 1, 0])
word_3 = np.array([1, 1, 0])
word_4 = np.array([0, 0, 1])

word_1, word_2, word_3, word_4

(array([1, 0, 0]), array([0, 1, 0]), array([1, 1, 0]), array([0, 0, 1]))

**2. Generate the weights matrices**

These weights will be multiplied by the word embeddings to generate the Queries, Keys and Values.

In [75]:
W_Q = np.random.randint(3, size=(3, 3))
W_K = np.random.randint(3, size=(3, 3))
W_V = np.random.randint(3, size=(3, 3))

W_Q, W_K, W_V

(array([[2, 0, 2],
        [2, 0, 0],
        [2, 1, 2]]),
 array([[2, 2, 2],
        [0, 2, 1],
        [0, 1, 1]]),
 array([[1, 1, 0],
        [0, 1, 1],
        [0, 0, 0]]))

### Compute the Attention

1. First, we calculate the Query, Key, Value for each word:

    - queries[i] = words[i] @ W_Query
    - keys[i] = words[i] @ W_Keys
    - values[i] = words[i] @ W_Values

2. Next, we score each word by taking the dot product of its query againts all the keys:

    - scores[i] = [ np.dot(queries[i] * keys[0]), np.dot(queries[i] * keys[1]), ... ]

**Calculate the Query, Key and Value for each word**

In [77]:
query_1 = word_1 @ W_Q
key_1 = word_1 @ W_K
value_1 = word_1 @ W_V

query_2 = word_2 @ W_Q
key_2 = word_2 @ W_K
value_2 = word_2 @ W_V

query_3 = word_3 @ W_Q
key_3 = word_3 @ W_K
value_3 = word_3 @ W_V

query_4 = word_4 @ W_Q
key_4 = word_4 @ W_K
value_4 = word_4 @ W_V

query_1, key_1, value_1

(array([2, 0, 2]), array([2, 2, 2]), array([1, 1, 0]))

**Score Values**

Now, considering each word,we score its query vector againts all the key vectors. 

In [93]:
matrix1 = np.array([[1, 2], [3, 4]])
matrix2 = np.array([[5, 6], [7, 8]])

matrix1 @ matrix2

array([[19, 22],
       [43, 50]])

In [98]:
# Scoring the first query vector against all vectors
scores = query_1 @ key_1, query_1 @ key_2, query_1 @ key_3, query_1 @ key_4
scores = np.array(scores)
scores

array([ 8,  2, 10,  2])

**Apply softmax to the scores**

The softmax function transform the attention scores to probabilities making it easier to interpret.

In [79]:
# To keep the gradient stable, we divide the scores by the square root of the dimension of the key vectors
d = np.sqrt(key_1.shape[0])
weights = softmax(scores / d)
weights

array([0.23608986, 0.00738988, 0.74913039, 0.00738988])

**Compute the weighted sum of weights and values**

Now, we use the attention weights to get a weighted sum of the Value vector. The weighted sum gives more importance to values that have higher attention weights.
This allow the model to focus on relevant parts of the input data.

*The weighted sum is nothing other than the context vector, whichi  represents aggregated information from the input data.*

In [87]:
attention = (weights[0] * value_1) + (weights[1] * value_2) + (weights[2] * value_3) + (weights[3] * value_4)
attention

array([0.98522025, 1.74174051, 0.75652026])

That's it! This  context vectors will be used as an input to the subsequent layers of the Deep Learning model.

### Putting it all together

Now, let's put all the pieces together to build a complete attention model for all the 4 words.

In [131]:
from numpy import array
from numpy import random
from numpy import dot
from numpy.random import randint
from scipy.special import softmax
np.random.seed(42)

# encoder representations of four different words
word_1 = array([1, 0, 0])
word_2 = array([0, 1, 0])
word_3 = array([1, 1, 0])
word_4 = array([0, 0, 1])

# stacking the word embeddings into a single array
words = array([word_1, word_2, word_3, word_4])

# generating the weight matrices
W_Q = randint(3, size=(3, 3))
W_K = randint(3, size=(3, 3))
W_V = randint(3, size=(3, 3))

# generating the queries, keys and values
Q = words @ W_Q
K = words @ W_K
V = words @ W_V

# scoring each query vector against all key vectors
scores = Q @ K.T

# computing the weights by a softmax operation
weights = softmax(scores / K.shape[1] ** 0.5, axis=1)

# computing the attention by a weighted sum of the value vectors
attention = weights @ V

print(attention)

[[0.98522025 1.74174051 0.75652026]
 [0.90965265 1.40965265 0.5       ]
 [0.99851226 1.75849334 0.75998108]
 [0.99560386 1.90407309 0.90846923]]


## References

- https://machinelearningmastery.com/the-attention-mechanism-from-scratch/
- https://www.cloudskillsboost.google/course_sessions/3962322/quizzes/383837


- The following prompts were used with ChatGPT
    - How does the attention mechanism work?
    - Is the weighted sum the context vector?
    - Concretely what represent the Query, Value and Key vectors?

Answers were carefully reviewed and edited for clarity.